# Build Belief Network with PGMPY

This notebook contains two simple examples to show how to use *PGMPY* package, which is a *Python* library for *Probabilistic Graphical Models*. You can access all related resources through its [official website](http://pgmpy.org/). You can install the package as simple as:

```bash
>> pip3 install pgmpy
```

However, in my system, there is a dependent package named 'wrapt' not installed in this problem. I need install it specificly.

```bash
>> pip3 install wrapt
```

Then you can access classes defined in package through keyword **import** in *Python*, like:

```python
from pgmpy.models import BayesianModel
```

Which can import classes and methods related to Bayesian models, which is also our major topic here.

## Problem One : Inspector Clouseau

*This is an example comes from book of Barber (Example 2 in Chapater 1)*

Inspector Clouseau arrives at the scene of a crime. The victim lies dead in the room and the inspector quickly ﬁnds the murder weapon, a Knife (K). The Butler (B) and Maid (M) are his main suspects. The inspector has a prior belief of 0.8 that the Butler is the murderer, and a prior belief of 0.2 that the Maid is the murderer. These probabilities are independent in the sense that p(B, M) = p(B)p(M). (It is possible that both the Butler and the Maid murdered the victim or neither). The inspector’s prior criminal knowledge can be formulated mathematically as follows:

$$
\begin{align}
P(B) & = 0.8 \\
P(M) & = 0.2 \\
P(M, B) & = P(M)P(B)
\end{align}
$$

Besides, Clouseau also has estimations that:

$$
\begin{align}
P(K | \bar{B}, \bar{M}) & = 0.3 \\
P(K | \bar{B}, M) & = 0.2 \\
P(K | B, \bar{M}) & = 0.6 \\
P(K | B, M) & = 0.1
\end{align}
$$

What is the probability that the Butler is the murderer? (Remember that it might be that neither is the murderer).

For this problem, we can get help from **pgmpy** package:

In [ ]:
from pgmpy.models import BayesianModel as bysmodel
from pgmpy.factors.discrete import TabularCPD as tcpd

### Build Up Network

In **pgmpy**, Bayesian network is specified by variables and connection between them. For building up a functional Bayesian model, you need describe the structure of network by claiming links in the corresponding directed graph, and assign conditional probabilities (or prior) to each variable.

In [ ]:
# define model with specifying connections between random variables
model = bysmodel([('B', 'K'), ('M', 'K')]);

# define prior with TabularCPD (TABULAR Conditional Probability Distribution) without condition
priorB = tcpd(variable='B', variable_card=2, values=[[0.2, 0.8]])
priorM = tcpd(variable='M', variable_card=2, values=[[0.8, 0.2]])

# define conditional probability of 'K' and 'B' with 'M'
cpdK = tcpd(variable='K', variable_card=2, 
            evidence=['B', 'M'], evidence_card=[2, 2],
            values=[[0.7, 0.8, 0.4, 0.9], 
                    [0.3, 0.2, 0.6, 0.1]]
           )

# add probabilities to model
model.add_cpds(priorB, priorM, cpdK)

After we built up our model, a simple way to check minor mistakes is function *check_model*. It will tell you whether or not your model fullfilled the axioms.

In [ ]:
# check structure and values consistency with Bayesian Model
model.check_model()

You also can let it show you conditional probability in tables. Let's compare it with our setup:

|           | B = false, M = false | B = false, M = true | B = true, M = false | B = true, M = true |
| --------- | -------------------- | ------------------- | ------------------- | ------------------ |
| K = false | 0.7 | 0.8 | 0.4 | 0.9 |
| K = true  | 0.3 | 0.2 | 0.6 | 0.1 |

In [ ]:
print(model.get_cpds('K'))

### Bayesian Inference

In [ ]:
from pgmpy.inference import VariableElimination as proc

In theory:

$$
\begin{align}
P(B|K) & = \sum_{M} P(B,M|K) \\
       & = \sum_{M} \dfrac{P(B,M,K)}{P(K)} \\
       & = \dfrac{P(B) \sum_{M} P(K|B,M)P(M)}{\sum_{B} P(B) \sum_{M} P(K|B,M) P(M)} \\
       & = \dfrac{0.8 \times (0.1 \times 0.2 + 0.6 \times 0.8)}{0.8 \times (0.1 \times 0.2 + 0.6 \times 0.8) + 0.2 \times (0.2 \times 0.2 + 0.3 \times 0.8)} \\
       & \approx 0.877
\end{align}
$$

Now, let's solve it with **pgmpy**:

In [ ]:
infer = proc(model)
print(infer.query(['B'], evidence={'K' : 1}) ['B'])

Simple, right?

## Problem Two : Poisson Distribution

The Poisson distribution has the following form

$$
p(n|\lambda) = \frac{1}{n!} \lambda^n e^{-\lambda}
$$

where λ > 0 is the rate of the Poisson process, and n is the number of observations per unit of time, n = {1, 2, . . .}. To derive the posterior distribution, we must deﬁne a prior. A natural choice is the gamma distribution distribution, which is a conjugate prior for the Poisson distribution

$$
\dfrac{\beta^\alpha}{\Gamma(\alpha)} \lambda^{\alpha - 1} e^{-\beta \lambda}
$$

where α is the shape parameter and β is the inverse scale parameter. A special case of the gamma is the exponential, Gamma(α = 1, β)

$$
\beta e^{-\beta \lambda}
$$

a) Derive the posterior distribution p(λ|n, T) where T is the total observation time, and λ is speciﬁed in terms of events per second.

the number of events $n$ that happened in an unit time follows :
$$p(n|\lambda) = \dfrac{1}{n!} \lambda^n e^{-\lambda}$$

Considering the condition that $n$ in a particular time interval $T$, we have :
$$p(n|\lambda,T) = \dfrac{1}{n!} (\lambda T)^n e^{-\lambda T}$$

We assign $\Gamma$ distribution as the **Prior** of $\ \lambda$ :
$$p(\lambda|\alpha,\beta) = \dfrac{\beta^{\alpha}}{\Gamma(\alpha)} \lambda^{\alpha-1} e^{-\beta \lambda}$$

Utilizing Bayes Rules, we can gradually get $Posterior$ of $\ \lambda$ :

$$
\begin{array}{lll}
	p(\lambda|n,\alpha,\beta,T) 
		& = & \dfrac{p(n|\lambda,\alpha,\beta,T) p(\lambda|\alpha,\beta,T)}{p(n|\alpha,\beta,T)} \\
	    & = & \dfrac{p(n|\lambda,T)p(\lambda|\alpha,\beta)}{\int_{\lambda=0}^{\inf} p(n|\lambda,T) p(\lambda|\alpha,\beta) d\lambda} \\
	    & = & \dfrac{\frac{1}{n!}(\lambda T)^n e^{-\lambda T} \frac{\beta^{\alpha}}{\Gamma(\alpha)} \lambda^{\alpha - 1} e^{-\beta \lambda}}{\int_{\lambda = 0}^{\inf} \frac{1}{n!}(\lambda T)^n e^{-\lambda T} \frac{\beta^{\alpha}}{\Gamma(\alpha)} \lambda^{\alpha - 1} e^{-\beta \lambda} d\lambda} \\
	    & = & \dfrac{\lambda^{(\alpha+n-1)} e^{-(\beta+T)\lambda}}{\int_{\lambda=0}^{\inf} \lambda^{(\alpha+n-1)} e^{-(\beta+T)\lambda} d\lambda} \\
	    & = & \dfrac{(\beta+T)^{(\alpha+n)}}{\Gamma(\alpha+n)} \lambda^{(\alpha+n-1)} e^{-(\beta+T)\lambda}
\end{array}
$$

There are two independence used in the deduction : between $\alpha$, $\beta$, and $n$; and between $\lambda$ and $T$. 

The last step is based on the definition of $\Gamma()$ function :
$$\Gamma(\alpha) = \int_{t=0}^{\inf} e^{-t} t^{\alpha-1} dt$$

While, $\int_{t=0}^{\inf} e^{-\beta t} t^{\alpha-1} dt$ can be transformed to $\int_{\mu=0}^{\inf} \frac{1}{\beta^\alpha} e^{-\mu} \mu^{\alpha-1} d\mu$ by variable replacement $\mu = \beta t$. Thus,
$$\int_{t=0}^{\inf} e^{-\beta t} t^{\alpha-1} dt = \dfrac{\Gamma(\alpha)}{\beta^\alpha}$$

b) Plots the posterior distribution given the following event times

```
0.53, 0.65, 0.91, 1.19, 1.30, 1.33, 1.90, 2.01, 2.48
```

Turn in plots of your belief about λ (i.e. the posterior distribution) given the above data at observation times T = {0, 0.5, 1, 5}. Label your axes, and make sure each plot has a clear title.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

Unfortunately, **pgmpy** package cannot help use in continuous variable inference in Bayesian models. However, it can be useful in *Markov Random Field* with *Sum-Product* algorithm. Here, we'll draw the plot based on our theoretical result above.

At very beginning, let's define a *Gamma* function:

$$
\Gamma(x, \alpha, \beta, n, T) = \dfrac{(\beta+T)^{(\alpha+n)}}{\Gamma(\alpha+n)} x^{(\alpha+n-1)} e^{-(\beta+T) x}
$$

In [ ]:
def mygamma(x, alpha, beta, n, T):
    a = alpha + n
    b = beta + T
    return b**a * x**(a-1) * math.e**(-b*x) / math.gamma(a)

Make given data a list for calculation:

In [ ]:
data = [0.53, 0.65, 0.91, 1.19, 1.30, 1.33, 1.90, 2.01, 2.48]

Then, setup parameters with our prior $\alpha = \beta = 1$:

In [ ]:
alpha = 2
beta  = 3

Draw posterior $p(\lambda|n, T)$ by counting at each time point:

In [ ]:
x = np.linspace(0, 5, 500)
for T in range(0,4):
    n = len([t for t in data if t <= T])
    plt.plot(x, mygamma(x, alpha, beta, n, T))

plt.grid()

Find the best estimation by searching for maxium value in posterior distribution:

In [ ]:
n = len(data)
T = math.ceil(data[-1])
# calculate posterior
posteriorDist = list(mygamma(x, alpha, beta, n, T))
# find maximum
print("Best Estimation of λ : ", x[posteriorDist.index(max(posteriorDist))])